In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
from matplotlib.ticker import FuncFormatter
import warnings
warnings.filterwarnings('ignore')
import re

In [151]:
df = pd.read_csv('SraRunTable.csv', index_col=False)
total_bytes = df['Bytes'].sum()
print(f"Общий объем данных: {total_bytes/1e12:.4f} TB")

Общий объем данных: 2.7105 TB


### Чистим от 16S rRNA было 2807 - стало 1707

In [140]:
deleted_rows = df[df['target_gene (exp)'] == '16S rRNA']
df = df[df['target_gene (exp)'] != '16S rRNA']
#df
#deleted_rows

### Убираем запуски с аномально большим количеством баз (больше 30 гигабаз и весят по 50-150 гигабайт) (пока не убираем)

In [141]:
#df = df[df['Bases'].apply(lambda x: x<30000000000)]
#df

### Удаляем хозяев мышь и выдру

In [142]:
df = df[(df['HOST'] != 'Mus musculus') & (df['HOST'] !='southern_sea_otter')]
#df

### Удаляем проект PRJNA1214429 с 311 архивами, в которых все прочтения бактериальные и PRJNA665586 (4 архива) просто мазок из пещеры - вирусных прочтений нет.

In [143]:
df = df[(df['BioProject'] != 'PRJNA1214429') & (df['BioProject'] != 'PRJNA665586')]
#df

### Удаляем проекты, где организм бактерии

In [ ]:

pattern = r'(Escherichia coli|Lactobacillus|Lactococcus sp\.|Escherichia marmotae|Clostridium sp\.|Enterobacteriaceae bacterium|Streptomyces sp\.|Staphylococcus nepalensis)'

mask = ~df['Organism'].str.contains(pattern, case=False, na=False, regex=True)
filtered_df = df[mask]

removed = df[~mask]
print(f"Удалено {len(removed)} строк:")
#print(removed['Organism'].unique())
df = filtered_df

Удалено 130 строк:


### Убираем пустые столбцы. В итоге остается 1189 архивов

In [145]:
df = df.dropna(axis=1, how='all')
#df.to_csv('sra_filtered.csv', index=False)

In [146]:
#df['tissue'].value_counts()
#df['HOST'].value_counts()
#df['Organism'].value_counts()
#df.loc[df['BioProject'] == 'PRJNA665586']
#df.loc[df['HOST'] == 'Environment']
#df.loc[df['isolation_source'] == 'forest']

### Данные весят 2.58 терабайт

In [155]:
df = pd.read_csv('sra_filtered.csv')
total_bytes = df['Bytes'].sum()
print(f"Общий объем данных: {total_bytes/1e12:.4f} TB")

Общий объем данных: 2.5857 TB


In [165]:
print("\nБольшие файлы:")
top_large = df.nlargest(10, 'Bytes')[['Run', 'Bytes', 'Assay Type', 'Organism']].copy()
top_large['Size_GB'] = top_large['Bytes'] / 1e9
top_large['Size_TB'] = top_large['Bytes'] / 1e12
print(top_large[['Run', 'Size_GB', 'Size_TB', 'Assay Type', 'Organism']])


Большие файлы:
             Run     Size_GB   Size_TB Assay Type            Organism
733  SRR23314745  161.519252  0.161519    RNA-Seq      bat metagenome
736  SRR23314746  123.813281  0.123813    RNA-Seq      bat metagenome
735  SRR23314743  100.071383  0.100071    RNA-Seq      bat metagenome
737  SRR23314747   61.614715  0.061615    RNA-Seq      bat metagenome
740  SRR23314751   57.810831  0.057811    RNA-Seq      bat metagenome
739  SRR23314750   53.888778  0.053889    RNA-Seq      bat metagenome
198  SRR19599582   34.619077  0.034619    RNA-Seq  Eonycteris spelaea
738  SRR23314749   34.270534  0.034271    RNA-Seq      bat metagenome
429  SRR14860617   33.661649  0.033662        WGS      gut metagenome
197  SRR19599581   33.630739  0.033631    RNA-Seq  Eonycteris spelaea


In [166]:
categorical_cols = ['Assay Type', 'LibraryLayout', 'LibrarySource', 'Platform', 'geo_loc_name_country', 'geo_loc_name_country_continent', 'sample_type']
for col in categorical_cols:
    if col in df.columns:
        print(f"\n{col}:")
        print(f"  Количество уникальных значений: {df[col].nunique()}")
        print(f"  Топ-10 значений: {df[col].value_counts().head(10).to_dict()}")


Assay Type:
  Количество уникальных значений: 3
  Топ-10 значений: {'RNA-Seq': 576, 'WGS': 498, 'OTHER': 115}

LibraryLayout:
  Количество уникальных значений: 2
  Топ-10 значений: {'PAIRED': 1100, 'SINGLE': 89}

LibrarySource:
  Количество уникальных значений: 7
  Топ-10 значений: {'METAGENOMIC': 797, 'METATRANSCRIPTOMIC': 186, 'GENOMIC': 108, 'VIRAL RNA': 47, 'TRANSCRIPTOMIC': 43, 'TRANSCRIPTOMIC SINGLE CELL': 4, 'OTHER': 4}

Platform:
  Количество уникальных значений: 7
  Топ-10 значений: {'ILLUMINA': 1166, 'OXFORD_NANOPORE': 7, 'DNBSEQ': 6, 'PACBIO_SMRT': 5, 'CAPILLARY': 3, 'ION_TORRENT': 1, 'GENEMIND': 1}

geo_loc_name_country:
  Количество уникальных значений: 38
  Топ-10 значений: {'Ghana': 221, 'China': 171, 'Panama': 101, 'Spain': 99, 'Switzerland': 70, 'Singapore': 62, 'USA': 43, 'uncalculated': 41, 'Australia': 38, 'South Korea': 31}

geo_loc_name_country_continent:
  Количество уникальных значений: 7
  Топ-10 значений: {'Asia': 308, 'Europe': 268, 'Africa': 261, 'North Ame